In [65]:
import os
import datetime
from fredapi import Fred
from dotenv import load_dotenv
from pymongo import MongoClient
import pandas as pd

load_dotenv(".env")
FRED_API_KEY = os.getenv("API_KEY")
fred = Fred(api_key=FRED_API_KEY)
client = MongoClient('localhost', 27017)
db = client.project2_data
collection = db.project2_collection

In [ ]:
# Define your series dictionary, start and end dates as before
series_dict = {
    'bonds2tr': ('DGS2', 'M'),
    'gdpworld': ('NYGDPMKTPCDWLD', 'A'),
    'gdp': ('GDP', 'Q'),
    'recession': ('JHDUSRGDPBR', 'Q'),
    'bond10tr': ('DGS10', 'M'),
    'ppi': ('PPIACO', 'M'),
    'cpi': ('CPIAUCSL', 'M'),
    'unrate': ('UNRATE', 'M'),
    'debt': ('GFDEBTN', 'Q'),
    'fedrate': ('FEDFUNDS', 'M'),
    'm0': ('BOGMBASE', 'M'),
    'm3': ('MABMM301USM189S', 'M'),
    'cbasstogdp': ('DDDI06USA156NWDB', 'A'),
    'resins': ('TOTRESNS', 'M'),
    'oil': ('WTISPLC', 'M'),
    'indpro': ('INDPRO', 'M'),
    'houses': ('MSPUS', 'Q'),
    'cp': ('CP', 'Q')
}

# Define dates
start_date = datetime.datetime(1970, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

# Initialize DataFrames
dfs = {'M': pd.DataFrame(), 'Q': pd.DataFrame(), 'A': pd.DataFrame()}

# Function to fetch and merge series data
def fetch_and_merge(df, var_name, series_id, start, end):
    series_data = fred.get_series(series_id)
    series_data = series_data.loc[start_date:end_date]
    series_df = series_data.to_frame(name=var_name)
    series_df.index.name = 'date'
    return df.join(series_df, how='inner') if not df.empty else series_df


# Loop through each series
for var_name, (series_id, freq) in series_dict.items():
    dfs[freq] = fetch_and_merge(dfs[freq], var_name, series_id, start_date, end_date)

In [67]:
dfs['M'] = dfs['M'].reset_index()
data_to_insert = dfs['M'].to_dict(orient='records')

In [ ]:
collection.insert_many(data_to_insert)

In [38]:
from pymongo import MongoClient
import pandas as pd
import os
import pathlib
from dotenv import load_dotenv
import datetime
from fredapi import Fred

# Determine paths and load .env file
env_path = pathlib.Path('../.env')
load_dotenv(dotenv_path=env_path)
FRED_API_KEY = os.getenv("FRED_API_KEY")
fred = Fred(api_key=FRED_API_KEY)

client = MongoClient('localhost', 27017)
db = client.project2_data
collection = db.project2_collection

# Define your series dictionary, start and end dates as before
series_dict = {
    'gdpworld': ('NYGDPMKTPCDWLD', 'A'),
    'gdp': ('GDP', 'Q'),
    'recession': ('JHDUSRGDPBR', 'Q'),
    'bonds10tr': ('DGS10', 'D'),
    'bonds2tr': ('DGS2', 'D'),
    'ppi': ('PPIACO', 'M'),
    'cpi': ('CPIAUCSL', 'M'),
    'unrate': ('UNRATE', 'M'),
    'debt': ('GFDEBTN', 'Q'),
    'fedrate': ('FEDFUNDS', 'M'),
    'm0': ('BOGMBASE', 'M'),
    'm3': ('MABMM301USM189S', 'M'),
    'cbasstogdp': ('DDDI06USA156NWDB', 'A'),
    'resins': ('TOTRESNS', 'M'),
    'oil': ('WTISPLC', 'M'),
    'indpro': ('INDPRO', 'M'),
    'houses': ('MSPUS', 'Q'),
    'cp': ('CP', 'Q')
}

# Define dates
start_date = datetime.datetime(1970, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

# Initialize DataFrames
dfs = {'D': pd.DataFrame(), 'M': pd.DataFrame(), 'Q': pd.DataFrame(), 'A': pd.DataFrame()}


def fetch_and_merge(df, var_name, series_id, start, end):
    series_data = fred.get_series(series_id)
    series_data = series_data.loc[start_date:end_date]
    series_df = series_data.to_frame(name=var_name)
    series_df.index.name = 'date'
    return df.join(series_df, how='outer') if not df.empty else series_df


# Loop through each series
for var_name, (series_id, freq) in series_dict.items():
    dfs[freq] = fetch_and_merge(
        dfs[freq], var_name, series_id, start_date, end_date)



In [58]:
df = {}
df['D_M'] = dfs['D'].resample('M').last()
df['M_M'] = dfs['M'].resample('M').last()
df['Q_M'] = dfs['Q'].resample('M').ffill()
new_dates = pd.date_range(start=df['Q_M'].index.min(), end='2022-12-31', freq='M')
df['Q_M'] = df['Q_M'].reindex(new_dates).ffill()


In [72]:
data = pd.concat([df['D_M'], df['M_M'], df['Q_M']], axis=1)
data

,bonds10tr,bonds2tr,ppi,cpi,unrate,fedrate,m0,m3,resins,oil,indpro,gdp,recession,debt,houses,cp
1970-01-31,7.75,NaN,36.500,37.900,3.9,8.98,76400.0,5.896000e+11,28.9,3.35,37.9372,1051.200,1.0,372007.0,23900.0,55.323
1970-02-28,6.90,NaN,36.700,38.100,4.2,8.98,75200.0,5.863000e+11,27.9,3.35,37.9122,1051.200,1.0,372007.0,23900.0,55.323
1970-03-31,7.08,NaN,36.700,38.300,4.4,7.76,75300.0,5.873000e+11,27.5,3.35,37.8630,1051.200,1.0,372007.0,23900.0,55.323
1970-04-30,7.82,NaN,36.800,38.500,4.6,8.10,76300.0,5.884000e+11,28.1,3.35,37.7656,1067.375,1.0,370094.0,24400.0,55.194
1970-05-31,7.95,NaN,36.800,38.600,4.8,7.95,76600.0,5.915000e+11,27.9,3.35,37.7216,1067.375,1.0,370094.0,24400.0,55.194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,3.15,3.45,269.546,295.320,3.7,2.33,5582200.0,2.165990e+13,3305.9,93.67,103.2344,25994.639,0.0,30928912.0,468000.0,3013.486
2022-09-30,3.83,4.22,267.898,296.539,3.5,2.56,5410900.0,2.152520e+13,3131.4,84.26,103.5283,25994.639,0.0,30928912.0,468000.0,3013.486
2022-10-31,4.10,4.51,265.061,297.987,3.7,3.08,5339600.0,2.143270e+13,3055.7,87.55,103.4114,26408.405,0.0,31419689.0,479500.0,2850.085
2022-11-30,3.68,4.38,263.157,298.598,3.6,3.78,5418700.0,2.139870e+13,3126.2,84.37,103.0707,26408.405,0.0,31419689.0,479500.0,2850.085


In [71]:
data = data.reset_index()
data_to_insert = data.to_dict(orient='records')
collection.insert_many(data_to_insert)

In [73]:
for document in collection.find():
    print(document)

{'_id': ObjectId('652bc9f905827a03bf415171'), 'index': datetime.datetime(1970, 1, 31, 0, 0), 'bonds10tr': 7.75, 'bonds2tr': nan, 'ppi': 36.5, 'cpi': 37.9, 'unrate': 3.9, 'fedrate': 8.98, 'm0': 76400.0, 'm3': 589600000000.0, 'resins': 28.9, 'oil': 3.35, 'indpro': 37.9372, 'gdp': 1051.2, 'recession': 1.0, 'debt': 372007.0, 'houses': 23900.0, 'cp': 55.323}
{'_id': ObjectId('652bc9f905827a03bf415172'), 'index': datetime.datetime(1970, 2, 28, 0, 0), 'bonds10tr': 6.9, 'bonds2tr': nan, 'ppi': 36.7, 'cpi': 38.1, 'unrate': 4.2, 'fedrate': 8.98, 'm0': 75200.0, 'm3': 586300000000.0, 'resins': 27.9, 'oil': 3.35, 'indpro': 37.9122, 'gdp': 1051.2, 'recession': 1.0, 'debt': 372007.0, 'houses': 23900.0, 'cp': 55.323}
{'_id': ObjectId('652bc9f905827a03bf415173'), 'index': datetime.datetime(1970, 3, 31, 0, 0), 'bonds10tr': 7.08, 'bonds2tr': nan, 'ppi': 36.7, 'cpi': 38.3, 'unrate': 4.4, 'fedrate': 7.76, 'm0': 75300.0, 'm3': 587300000000.0, 'resins': 27.5, 'oil': 3.35, 'indpro': 37.863, 'gdp': 1051.2, 're